# Bird Data

### Import libraries and configure environment

In [6]:
import requests
import csv
import pandas as pd
import os

from dotenv import load_dotenv
load_dotenv()
api_key = os.environ.get('EBIRD_API_KEY')

### GET Requests to eBird API

In [7]:
# Fetches species codes for all recorded observations in PA
pa_species_url = 'https://api.ebird.org/v2/product/spplist/US-PA'
pa_species_params = {'key': os.environ.get('EBIRD_API_KEY')}
pa_species = requests.get(pa_species_url, params=pa_species_params)

# Handle potential JSONDecodeError
try:
    pa_species_data = pa_species.json()
except requests.exceptions.JSONDecodeError:
    print("Error decoding JSON response for pa_species:")
    print(pa_species.text)
    exit(1)

# Convert species codes to nested list format
pa_species_data_nested = [[code] for code in pa_species_data]

# Write species codes response to a CSV file
with open('pa_species.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Index', 'Species Code'])  # Write header
    for index, species_code in enumerate(pa_species_data, start=1):
        writer.writerow([index, species_code])


In [8]:
# Fetches all recent recorded observations in PA
recent_observations_url = 'https://api.ebird.org/v2/data/obs/US-PA/recent'
recent_observations_params = {'key': os.environ.get('EBIRD_API_KEY')}
recent_observations = requests.get(recent_observations_url, params=recent_observations_params)

# Handle potential JSONDecodeError
try:
    recent_observations_data = recent_observations.json()
except requests.exceptions.JSONDecodeError:
    print("Error decoding JSON response for recent_observations:")
    print(recent_observations.text)
    exit(1)

# Extract keys from the first observation to use as CSV header
header = list(recent_observations_data[0].keys())

# Write recent observations response to a CSV file
with open('recent_observations.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Index'] + header)  # Write header
    for index, observation in enumerate(recent_observations_data, start=1):
        writer.writerow([index] + list(observation.values()))